# Arxiv Chatter

First we we will create a program that will find the relevant papers about a subject.

In this case let us talk about  `Large Language Models`

In [1]:
import arxiv
import pandas as pd
import urllib.request
import fitz
import re
import numpy as np
import tensorflow_hub as hub
import gradio as gr
import os
from sklearn.neighbors import NearestNeighbors
# Compability with Hugging Face models
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import torch
import gradio as gr
import re
# Compability with ChatGPT
import openai

In [2]:
topic='Large Language Models LLM'

In [3]:
import arxiv
search = arxiv.Search(
  query = topic,
  max_results = 10,
  sort_by = arxiv.SortCriterion.Relevance
)
for result in arxiv.Client().results(search):
  print(result.title)

Several categories of Large Language Models (LLMs): A Short Survey
On the Origin of LLMs: An Evolutionary Tree and Graph for 15,821 Large Language Models
llm-japanese-dataset v0: Construction of Japanese Chat Dataset for Large Language Models and its Methodology
TEST: Text Prototype Aligned Embedding to Activate LLM's Ability for Time Series
FATE-LLM: A Industrial Grade Federated Learning Framework for Large Language Models
Alphazero-like Tree-Search can Guide Large Language Model Decoding and Training
Instruct-Align: Teaching Novel Languages with to LLMs through Alignment-based Cross-Lingual Instruction
Defending Against Alignment-Breaking Attacks via Robustly Aligned LLM
Large Language Models are Effective Table-to-Text Generators, Evaluators, and Feedback Providers
mBLIP: Efficient Bootstrapping of Multilingual Vision-LLMs


## Building our Dataframe of our Topic in Arxiv

In [4]:
titles = [result.title for result in arxiv.Client().results(search)]
authors = [result.authors for result in arxiv.Client().results(search)]
summary = [result.summary for result in arxiv.Client().results(search)]
entry_id = [result.entry_id for result in arxiv.Client().results(search)]
#download_source = [result.download_source for result in arxiv.Client().results(search)]
#download_pdf = [result.download_pdf for result in arxiv.Client().results(search)]
pdf_url = [result.pdf_url for result in arxiv.Client().results(search)]
categories = [result.categories for result in arxiv.Client().results(search)]
comment = [result.comment for result in arxiv.Client().results(search)]
doi = [result.doi for result in arxiv.Client().results(search)]
published = [result.published for result in arxiv.Client().results(search)]

In [5]:
# Building the dataframe

In [6]:
df = pd.DataFrame({
    'title': titles,
    'authors': authors,
    'summary': summary,
    #'entry_id': entry_id,
    'pdf_url':pdf_url,
    'categories':categories,
    'published':published
    
})

In [7]:
df.insert(0, 'id', range(1, len(df) + 1))

In [8]:
print(f'{df.shape[0]} results were found.')

10 results were found.


In [9]:
df

,id,title,authors,summary,pdf_url,categories,published
0,1,Several categories of Large Language Models (L...,"[Saurabh Pahune, Manoj Chandrasekharan]",Large Language Models(LLMs)have become effecti...,http://arxiv.org/pdf/2307.10188v1,"[cs.CL, cs.LG]",2023-07-05 18:18:23+00:00
1,2,On the Origin of LLMs: An Evolutionary Tree an...,"[Sarah Gao, Andrew Kean Gao]","Since late 2022, Large Language Models (LLMs) ...",http://arxiv.org/pdf/2307.09793v1,"[cs.DL, cs.CL, I.2.1; H.5.0]",2023-07-19 07:17:43+00:00
2,3,llm-japanese-dataset v0: Construction of Japan...,"[Masanori Hirano, Masahiro Suzuki, Hiroki Sakaji]",This study constructed a Japanese chat dataset...,http://arxiv.org/pdf/2305.12720v1,"[cs.CL, cs.AI]",2023-05-22 04:59:33+00:00
3,4,TEST: Text Prototype Aligned Embedding to Acti...,"[Chenxi Sun, Yaliang Li, Hongyan Li, Shenda Hong]",This work summarizes two strategies for comple...,http://arxiv.org/pdf/2308.08241v1,"[cs.CL, cs.AI]",2023-08-16 09:16:02+00:00
4,5,FATE-LLM: A Industrial Grade Federated Learnin...,"[Tao Fan, Yan Kang, Guoqiang Ma, Weijing Chen,...","Large Language Models (LLMs), such as ChatGPT,...",http://arxiv.org/pdf/2310.10049v1,"[cs.LG, cs.AI]",2023-10-16 04:17:13+00:00
5,6,Alphazero-like Tree-Search can Guide Large Lan...,"[Xidong Feng, Ziyu Wan, Muning Wen, Ying Wen, ...",Large language models (LLMs) typically employ ...,http://arxiv.org/pdf/2309.17179v1,"[cs.LG, cs.AI, cs.CL]",2023-09-29 12:20:19+00:00
6,7,Instruct-Align: Teaching Novel Languages with ...,"[Samuel Cahyawijaya, Holy Lovenia, Tiezheng Yu...",Instruction-tuned large language models (LLMs)...,http://arxiv.org/pdf/2305.13627v1,"[cs.CL, cs.AI]",2023-05-23 02:51:34+00:00
7,8,Defending Against Alignment-Breaking Attacks v...,"[Bochuan Cao, Yuanpu Cao, Lu Lin, Jinghui Chen]","Recently, Large Language Models (LLMs) have ma...",http://arxiv.org/pdf/2309.14348v1,"[cs.CL, cs.AI, cs.CR, cs.LG]",2023-09-18 02:07:22+00:00
8,9,Large Language Models are Effective Table-to-T...,"[Yilun Zhao, Haowei Zhang, Shengyun Si, Linyon...",Large language models (LLMs) have shown remark...,http://arxiv.org/pdf/2305.14987v1,[cs.CL],2023-05-24 10:22:30+00:00
9,10,mBLIP: Efficient Bootstrapping of Multilingual...,"[Gregor Geigle, Abhay Jain, Radu Timofte, Gora...",Modular vision-language models (Vision-LLMs) a...,http://arxiv.org/pdf/2307.06930v2,"[cs.CV, cs.CL]",2023-07-13 17:51:58+00:00


In [10]:
df['pdf_url']

0    http://arxiv.org/pdf/2307.10188v1
1    http://arxiv.org/pdf/2307.09793v1
2    http://arxiv.org/pdf/2305.12720v1
3    http://arxiv.org/pdf/2308.08241v1
4    http://arxiv.org/pdf/2310.10049v1
5    http://arxiv.org/pdf/2309.17179v1
6    http://arxiv.org/pdf/2305.13627v1
7    http://arxiv.org/pdf/2309.14348v1
8    http://arxiv.org/pdf/2305.14987v1
9    http://arxiv.org/pdf/2307.06930v2
Name: pdf_url, dtype: object

# Step 1 - PDF Analizer


The first example that we will use in this notebookwill we  will take the  the most relevant


In [11]:
url = df['pdf_url'][0]

In [12]:
print(url)

http://arxiv.org/pdf/2307.10188v1


In [13]:
def download_pdf(url, output_path):
    urllib.request.urlretrieve(url, output_path)

In [14]:
downloaded_file = "document.pdf"

In [15]:
download_pdf(url, downloaded_file)

In [16]:
def pdf_to_text(path, start_page=1, end_page=None):
    doc = fitz.open(path)
    total_pages = doc.page_count

    if end_page is None:
        end_page = total_pages

    text_list = []

    for i in range(start_page-1, end_page):
        text = doc.load_page(i).get_text("text")
        text = preprocess(text)
        text_list.append(text)

    doc.close()
    return text_list

def preprocess(text):
    text = text.replace('\n', ' ')
    text = re.sub('\s+', ' ', text)
    return text


In [17]:
text_list = pdf_to_text(downloaded_file)

In [18]:
len(text_list)

26

In [19]:
def text_to_chunks(texts, word_length=150, start_page=1):
    text_toks = [t.split(' ') for t in texts]
    page_nums = []
    chunks = []
    
    for idx, words in enumerate(text_toks):
        for i in range(0, len(words), word_length):
            chunk = words[i:i+word_length]
            if (i+word_length) > len(words) and (len(chunk) < word_length) and (
                len(text_toks) != (idx+1)):
                text_toks[idx+1] = chunk + text_toks[idx+1]
                continue
            chunk = ' '.join(chunk).strip()
            chunk = f'[Page no. {idx+start_page}]' + ' ' + '"' + chunk + '"'
            chunks.append(chunk)
    return chunks

In [20]:
chunks= text_to_chunks(text_list)

In [21]:
parts = len(chunks)

In [22]:
chunks[:1]

['[Page no. 1] "Several categories of Large Language Models (LLMs): A Short Survey Saurabh Pahune 1,†,‡, Manoj Chandrasekharan 2 1 Cardinal Health, Dublin OH 43017, USA; Email: saurabh.pahune@cardinalhealth.com, Tel.:+1-901-691-7551 2 Email: manoj.c@memphis.edu Abstract: Large Language Models (LLMs) have become effective tools for natural language process- ing and have been used in many different ﬁelds. This essay offers a succinct summary of various LLM subcategories. The survey emphasizes recent developments and efforts made for various LLM kinds, including task-based ﬁnancial LLMs, multilingual language LLMs, biomedical and clinical LLMs, vision language LLMs, and code language models. The survey gives a general summary of the methods, attributes, datasets, transformer models, and comparison metrics applied in each category of LLMs. Furthermore, it highlights unresolved problems in the ﬁeld of developing chatbots and virtual assistants, such as boosting natural language processing, 

In [23]:
print("We have {} pieces of the article now".format(parts))

We have 68 pieces of the article now


Now above all the list of 41 pieces, we should reduce the amount of pieces, this is possible by usuing the Semantic Search.
Thie is great model used by Google that his the universal sentence encoder.

In [24]:
class SemanticSearch:
    def __init__(self):
        self.use = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
        self.fitted = False
    def fit(self, data, batch=1000, n_neighbors=5):
        self.data = data
        self.embeddings = self.get_text_embedding(data, batch=batch)
        n_neighbors = min(n_neighbors, len(self.embeddings))
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)
        self.nn.fit(self.embeddings)
        self.fitted = True
    def __call__(self, text, return_data=True):
        inp_emb = self.use([text])
        neighbors = self.nn.kneighbors(inp_emb, return_distance=False)[0]
        if return_data:
            return [self.data[i] for i in neighbors]
        else:
            return neighbors
    def get_text_embedding(self, texts, batch=1000):
        embeddings = []
        for i in range(0, len(texts), batch):
            text_batch = texts[i:(i+batch)]
            emb_batch = self.use(text_batch)
            embeddings.append(emb_batch)
        embeddings = np.vstack(embeddings)
        return embeddings

We summarize all the previous steps

In [25]:
recommender = SemanticSearch()

In [26]:
def load_recommender(path, start_page=1):
    global recommender
    texts = pdf_to_text(path, start_page=start_page)
    chunks = text_to_chunks(texts, start_page=start_page)
    recommender.fit(chunks)
    return 'Corpus Loaded.'

In [27]:
load_recommender(downloaded_file , start_page=1)

'Corpus Loaded.'

In [28]:
question="Give me summary of the abstract of the document"

In [29]:
DEFAULT_SYSTEM_PROMPT = """\Compose a comprehensive reply to the query using the search results given. 
Cite each reference using [ Page Number] notation (every result has this number at the beginning). 
Citation should be done at the end of each sentence. If the search results mention multiple subjects 
with the same name, create separate answers for each. Only include information found in the results and 
don't add any additional information. Make sure the answer is correct and don't output false content. 
If the text does not relate to the query, simply state 'Found Nothing'. Ignore outlier 
search results which has nothing to do with the question. Only answer what is asked. The 
answer should be short and concise.\
"""

In [30]:
def get_simple_prompt(question: str) -> str:
    topn_chunks = recommender(question)
    results = ""
    results += 'Search results:\n\n'
    for c in topn_chunks:
        results += c + '. \n\n' 
    message =  "\n\nQuery: {}".format(question) + " \n\n" + results + " \nAnswer:"
    texts = [f'<s>[INST] <<SYS>>\n{DEFAULT_SYSTEM_PROMPT}\n<</SYS>>\n\n']
    texts.append(f'{message} [/INST]')
    return ''.join(texts)

In [31]:
prompt=get_simple_prompt(question)

# Modeling 1 

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# Load the model and tokenizer
try:
    model_name
    model
    tokenizer
except NameError:
    print("Loading the model ...")
    model_name = "meta-llama/Llama-2-7b-chat-hf"
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)    
else:
    print("The model was loaded.")

The model was loaded.


In [34]:
# Define a base instruction for the model
base_instruction  = "Instructions: Compose a comprehensive reply to the query using the search results given. "\
              "Cite each reference using [Page Number] notation (every result has this number at the beginning). "\
              "Citation should be done at the end of each sentence. If the search results mention multiple subjects "\
              "with the same name, create separate answers for each. Only include information found in the results and "\
              "don't add any additional information. Make sure the answer is correct and don't output false content. "\
              "If the text does not relate to the query, simply state 'Found Nothing'. Ignore outlier "\
              "search results which has nothing to do with the question. Only answer what is asked. The "\
              "answer should be short and concise."\
              "Do not include the instructions in the answer."
# Define a list of sample texts that will be used as input along with the prompt
prompt = "What is the abstract"
results='[Page no. 1] "Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need Ashish Vaswani∗ Google Brain avaswani@google.com Noam Shazeer∗ Google Brain noam@google.com Niki Parmar∗ Google Research nikip@google.com Jakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research llion@google.com Aidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu Łukasz Kaiser∗ Google Brain lukaszkaiser@google.com Illia Polosukhin∗ ‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly"'
# Combine the base instruction and the user's prompt
combined_prompt = base_instruction + "\n\nQuery: {}".format(prompt) + " \n\n" + results + " \nAnswer:"
# Set the pad_token attribute of the tokenizer to use the end-of-sequence (eos) token as padding
tokenizer.pad_token = tokenizer.eos_token
# Create a batch of combined prompt
inputs = tokenizer([combined_prompt], return_tensors="pt", padding=True)
# Get the input_ids  from the tokenizer's output
input_ids = inputs["input_ids"]
# Generate text using the `model.generate()` function, providing input_ids

In [35]:
generated_text = model.generate(
    input_ids=input_ids,
    max_length=1000,
) 
# Decode and print the generated text without special tokens
generated_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
# Print the generated text
print(generated_text)

Instructions: Compose a comprehensive reply to the query using the search results given. Cite each reference using [Page Number] notation (every result has this number at the beginning). Citation should be done at the end of each sentence. If the search results mention multiple subjects with the same name, create separate answers for each. Only include information found in the results and don't add any additional information. Make sure the answer is correct and don't output false content. If the text does not relate to the query, simply state 'Found Nothing'. Ignore outlier search results which has nothing to do with the question. Only answer what is asked. The answer should be short and concise.Do not include the instructions in the answer.

Query: What is the abstract 

[Page no. 1] "Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need Ash